<a href="https://colab.research.google.com/github/hmh100298/MLP_from_Scratch/blob/main/MLP_NP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
np.random.seed(42)

In [3]:
class Layer:
  def __init__(self):
    pass

  def forward(self, input):
    return input

  def backward(self, input, grad_output): #  d loss / d x  = (d loss / d layer) * (d layer / d x)
    num_units = input.shape[1]   # so phan tu axis 1
    d_layer_d_input = np.eye(num_units) # eye: tao ma tran don vi 
    return np.dot(grad_output, d_layer_d_input)

class ReLU(Layer):
  def __init__(self):
    pass
  def forward(self, input):
        """Apply elementwise ReLU to [batch, input_units] matrix"""
        relu_forward = np.maximum(0,input)
        return relu_forward
    
  def backward(self, input, grad_output):
        """Compute gradient of loss w.r.t. ReLU input"""
        relu_grad = input > 0
        return grad_output*relu_grad
      
class Dense(Layer):
  def __init__(self, input_units, output_units, learning_rate=0.1):
    self.learning_rate = learning_rate
    self.weights = np.random(loc = 0.0, scale = np.sqrt(2/(input_units+ output_units)), size = (input_units,output_units))
    

In [22]:
input = np.array([[1, 2, 3], [-1, -2, 4]])
relu = np.maximum(0,input)
relu_grad = input > 0
print(input)
print(relu_grad)
print(relu_grad*relu)

[[ 1  2  3]
 [-1 -2  4]]
[[ True  True  True]
 [False False  True]]
[[1 2 3]
 [0 0 4]]
